In [1]:
import pandas as pd
import numpy as np

In [28]:
hpd_vio = pd.read_csv("Housing_Maintenance_Code_Violations.csv", low_memory=False, dtype={'BIN': 'str', 'BBL': 'str'})

In [29]:
hpd_vio_lite = hpd_vio.loc[:,[ 'BIN', 'BBL', 'InspectionDate']]

In [38]:
hpd_vio_lite['year'] = hpd_vio_lite['InspectionDate'].apply(lambda x: x[-4:])
hpd_vio_lite['year'] = hpd_vio_lite['year'].astype(int)
hpd_vio_lite_new = hpd_vio_lite[hpd_vio_lite['year']>=2014]
hpd_vio_lite_new.head()

,BIN,BBL,InspectionDate,year
30,2004901,2026740027,07/12/2018,2018
72,2011602,2030520018,10/15/2018,2018
157,1053175,1017160049,07/13/2018,2018
164,1055722,1018440025,10/05/2017,2017
174,1064716,1022170039,10/05/2018,2018


In [39]:
hpd_vio_lite_new = hpd_vio_lite_new.drop(columns=['InspectionDate'])
hpd_vio_lite_new = hpd_vio_lite_new.drop_duplicates(subset=['BIN', 'BBL', 'year'], keep='first')
hpd_vio_lite_new.head()

,BIN,BBL,year
30,2004901,2026740027,2018
72,2011602,2030520018,2018
157,1053175,1017160049,2018
164,1055722,1018440025,2017
174,1064716,1022170039,2018


In [40]:
len(hpd_vio_lite_new.index) 

403187

In [46]:
shd_clean = pd.read_csv("shd_clean.csv",  dtype={'bbl': 'str'})
shd_clean.head()

,bbl,tract_10,subsidy_flag,start_earliest,end_latest,start_earliest_time,end_latest_time
0,1000160015,36061031704,1,20000101,20301231,2000-01-01,2030-12-31
1,1000160020,36061031704,1,19990101,20291231,1999-01-01,2029-12-31
2,1000160200,36061031703,1,19990101,20291231,1999-01-01,2029-12-31
3,1000160210,36061031703,1,19980101,20281231,1998-01-01,2028-12-31
4,1000787510,36061001502,1,20070328,20381231,2007-03-28,2038-12-31


In [47]:
shd_clean = shd_clean[shd_clean['end_latest']>=20140101]
shd_clean['start_earliest']=shd_clean['start_earliest'].astype(str)
shd_clean['start_year'] = shd_clean['start_earliest'].apply(lambda x: x[0:4])
shd_clean.head()

,bbl,tract_10,subsidy_flag,start_earliest,end_latest,start_earliest_time,end_latest_time,start_year
0,1000160015,36061031704,1,20000101,20301231,2000-01-01,2030-12-31,2000
1,1000160020,36061031704,1,19990101,20291231,1999-01-01,2029-12-31,1999
2,1000160200,36061031703,1,19990101,20291231,1999-01-01,2029-12-31,1999
3,1000160210,36061031703,1,19980101,20281231,1998-01-01,2028-12-31,1998
4,1000787510,36061001502,1,20070328,20381231,2007-03-28,2038-12-31,2007


In [64]:
shd_clean_lite = shd_clean[['bbl','start_year']]

In [81]:
bbl_list = list(shd_clean_lite['bbl'])
count = 0
for unique_bbl in bbl_list:
    start_year = shd_clean_lite[shd_clean_lite['bbl'] == unique_bbl].iloc[0,1]
    start_year = int(start_year)
    temp_df = hpd_vio_lite_new[hpd_vio_lite_new['BBL'] == unique_bbl]
    num = num + 1
    if len(temp_df) > 0:
        temp_df = temp_df[temp_df['year']>=start_year]
        if len(temp_df) > 0:
            if count == 0:
                new_df = temp_df
                count = 1
            else:
                new_df = pd.concat([new_df, temp_df], axis = 0)
    

In [75]:
len(new_df)

36196

In [76]:
new_df['subsidies_flag'] = 1

In [79]:
new_df = new_df[new_df['year']<2019]
len(new_df)

33312

In [80]:
new_df.to_csv("subsidized_BBL.csv", index=False)

In [116]:
housing_projects = pd.read_csv("housing_projects_output.csv", dtype={'BIN': 'str', 'BBL': 'str'})

housing_projects = housing_projects[housing_projects['year']<2019]
len(housing_projects)

2674

In [124]:
hpd_vio_lite_new_2 = hpd_vio_lite_new
hpd_vio_lite_new_2['affordability_flag'] = 0
hpd_vio_lite_new_2.head()


,BIN,BBL,year,affordability_flag
30,2004901,2026740027,2018,0
72,2011602,2030520018,2018,0
157,1053175,1017160049,2018,0
164,1055722,1018440025,2017,0
174,1064716,1022170039,2018,0


In [138]:
count = 0
for i in range(0,len(housing_projects)):
    year = housing_projects.iloc[i,0]
    BBL = housing_projects.iloc[i,1]
    BIN = housing_projects.iloc[i,2]
    affordability = housing_projects.iloc[i,3]
    temp_df = hpd_vio_lite_new[hpd_vio_lite_new_2['BBL'] == BBL]
    if len(temp_df)> 0:
        if BIN > 0:
            temp_df = temp_df[temp_df['BIN'] == BIN]
        if len(temp_df)>0:
            temp_df = temp_df[temp_df['year'] > year]
            if len(temp_df) > 0:
                temp_df.iloc[:,3] = affordability
                if count == 0:
                    new_df = temp_df
                    count = 1
                else:
                    new_df = pd.concat([new_df, temp_df], axis = 0)
            

In [135]:
new_df = new_df[new_df['year'] <2019]
len(new_df)

2440

In [136]:
new_df

,BIN,BBL,year,affordability_flag
2238026,3029966,3012050028,2018,4
2343383,3030845,3012230053,2018,4
1916221,3049319,3017710005,2018,0
2308597,3029554,3011770030,2018,4
1512204,2097215,2029610025,2018,4
2481644,1062245,1020840049,2018,3
2256985,1062248,1020840053,2018,3
1397229,2009684,2029310013,2018,4
691494,2009657,2029290078,2018,4
3225175,2009779,2029350028,2018,4


In [137]:
new_df.to_csv("affordability_clean.csv", index = False)